In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'molten-unison-414815'

In [2]:
REGION = 'us-central1'
EXPERIMENT = '01'
SERIES = '01'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud'

# Data source for this series of notebooks: Described above
BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

In [3]:
from google.cloud import bigquery
from google.cloud import storage

In [4]:
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client(project = PROJECT_ID)

In [5]:
BUCKET = PROJECT_ID

In [6]:
file = f"{SERIES}/{EXPERIMENT}/data/{BQ_TABLE}.csv"

In [7]:
bucketDef = gcs.bucket(BUCKET)
if storage.Blob(bucket = bucketDef, name = file).exists(gcs):
    print(f'The file has already been created at: gs://{bucketDef.name}/{file}')
else:
    source = bigquery.TableReference.from_string(BQ_SOURCE)
    extract = bq.extract_table(source = source, destination_uris = [f'gs://{bucketDef.name}/{file}'])
    print('Creating the export ...')
    extract.result()
    print(f'Exported the table to: gs://{bucketDef.name}/{file}')

Creating the export ...
Exported the table to: gs://molten-unison-414815/01/01/data/fraud.csv


In [8]:
list(bucketDef.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}'))



[<Blob: molten-unison-414815, 01/01/data/fraud.csv, 1708450699481579>]

In [9]:
print(f'Review the files in the console here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID}/{SERIES};tab=objects&project={PROJECT_ID}')


Review the files in the console here:
https://console.cloud.google.com/storage/browser/molten-unison-414815/01;tab=objects&project=molten-unison-414815


In [10]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

iris_dataset


In [11]:
ds = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
ds.location = REGION
ds.labels = {'experiment': f'{EXPERIMENT}'}
ds = bq.create_dataset(dataset = ds, exists_ok = True)

In [12]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

fraud
iris_dataset


In [13]:
from google.cloud.exceptions import NotFound
try:
    table = bq.get_table(f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
    if table:
        print(f'The table already exists: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
except NotFound as error:
    print(f'Creating Table ...')
    destination = bigquery.TableReference.from_string(f"{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}")
    job_config = bigquery.LoadJobConfig(
        write_disposition = 'WRITE_TRUNCATE',
        source_format = bigquery.SourceFormat.CSV,
        autodetect = True,
        labels = {'experiment': f'{EXPERIMENT}'}
    )
    job = bq.load_table_from_uri(f"gs://{bucketDef.name}/{file}", destination, job_config = job_config)
    job.result()
    print(f'Finished creating table: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')

Creating Table ...
Finished creating table: molten-unison-414815.fraud.fraud


In [14]:
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` TABLESAMPLE SYSTEM (1 PERCENT)
#LIMIT 5
"""
bq.query(query = query).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,150336,2.030851,-0.235559,-2.804940,-0.547376,2.441730,3.277629,-0.652959,0.843118,0.542395,...,-0.058377,-0.092950,0.203618,0.642094,-0.105975,0.502090,-0.024706,-0.040995,0.78,0
1,84068,-0.679466,1.334849,1.367987,0.856085,0.000189,-0.701735,0.582134,-0.003885,-0.741823,...,0.077499,0.311384,-0.095417,0.413387,-0.212580,-0.457638,-0.250283,-0.139162,0.78,0
2,122091,2.141596,0.077735,-2.104923,0.061232,0.368903,-1.825884,0.748172,-0.564815,0.354072,...,0.131269,0.490071,-0.035338,0.008649,0.415350,0.245709,-0.091809,-0.081979,0.78,0
3,136167,2.073311,0.265580,-1.745544,0.492756,0.286109,-1.427709,0.310907,-0.422277,0.505541,...,0.191552,0.767662,-0.043917,-0.156863,0.295408,-0.096150,0.002101,-0.031715,0.78,0
4,150523,2.032967,-0.387542,-0.505876,0.350922,-0.424833,-0.134032,-0.623411,-0.005357,1.481508,...,0.151034,0.737664,0.044700,-0.682962,-0.007459,-0.166488,0.047286,-0.044997,0.78,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142401,154599,0.667714,3.041502,-5.845112,5.967587,0.213863,-1.462923,-2.688761,0.677764,-3.447596,...,0.329760,-0.941383,-0.006075,-0.958925,0.239298,-0.067356,0.821048,0.426175,6.74,1
142402,90676,-2.405580,3.738235,-2.317843,1.367442,0.394001,1.919938,-3.106942,-10.764403,3.353525,...,10.005998,-2.454964,1.684957,0.118263,-1.531380,-0.695308,-0.152502,-0.138866,6.99,1
142403,34634,0.333499,1.699873,-2.596561,3.643945,-0.585068,-0.654659,-2.275789,0.675229,-2.042416,...,0.469212,-0.144363,-0.317981,-0.769644,0.807855,0.228164,0.551002,0.305473,18.96,1
142404,96135,-1.952933,3.541385,-1.310561,5.955664,-1.003993,0.983049,-4.587235,-4.892184,-2.516752,...,-1.998091,1.133706,-0.041461,-0.215379,-0.865599,0.212545,0.532897,0.357892,18.96,1


In [15]:
print(f"Direct Link To This Project In BigQuery:\nhttps://console.cloud.google.com/bigquery?project={PROJECT_ID}")


Direct Link To This Project In BigQuery:
https://console.cloud.google.com/bigquery?project=molten-unison-414815


In [16]:
query = f"""
SELECT Class
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""
df = bq.query(query = query).to_dataframe()

In [17]:
df['Class'].value_counts()


Class
0    284315
1       492
Name: count, dtype: Int64

In [18]:
query = f"""
CREATE TABLE IF NOT EXISTS `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`)
SELECT *,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits
FROM add_id
"""
job = bq.query(query = query)
job.result()

In [19]:
(job.ended-job.started).total_seconds()


7.577

In [20]:
if job.estimated_bytes_processed:
    print(f'{job.estimated_bytes_processed/1000000} MB')

70.632136 MB


In [21]:
query = f"""
SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
GROUP BY splits
"""
bq.query(query = query).to_dataframe()

,splits,Count,Percentage
0,TEST,28467,9.995190
1,TRAIN,228089,80.085461
2,VALIDATE,28251,9.919349


In [22]:
query = f"""
SELECT * 
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
LIMIT 5
"""
data = bq.query(query = query).to_dataframe()

In [23]:
data.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,73378,1.267971,-0.071333,0.008482,-0.392620,-0.027840,-0.015306,-0.182631,0.109003,0.065205,...,0.051971,-0.627543,0.144160,0.903631,-0.078328,-0.017881,0.0,0,7f272212-f1dc-4a6c-bad6-c325c2af4140,TEST
1,36686,-1.116637,0.766187,3.219085,1.051747,-0.047797,0.510289,0.417527,0.228175,-0.028570,...,-0.501597,0.543240,0.625938,-0.098797,-0.272474,-0.177326,0.0,0,26ec0caa-cede-486e-b0a4-aee6f4ee1fcf,TEST
2,80210,1.106933,0.208589,1.411978,2.514973,-0.564985,0.670207,-0.659376,0.326511,-0.145195,...,-0.009164,0.002618,0.316512,-0.040788,0.039501,0.022443,0.0,0,63b36024-7e35-403f-974a-cfc3a22ba40f,TEST
3,67511,-1.585593,1.084266,-0.181189,-0.375810,0.143916,-0.775504,-0.084983,0.913627,-0.070178,...,-0.163398,-0.239701,-0.368140,0.354910,0.300076,0.041391,0.0,0,d43217d5-ecaa-44d5-983f-c00e9bb3ce4e,TEST
4,130876,0.000664,1.480795,1.247668,4.298910,0.443660,2.511772,-1.312127,-2.199241,-2.065188,...,-0.146442,-1.522810,0.965808,0.230626,0.065651,0.177569,0.0,0,e11f652f-ba02-4620-b730-efbfde6ca2a2,TEST
